# Predict Sentiments And Store Them In Gold Table

In [0]:
%sql
select * from infosys.kamikaze_rtsa.gold_table_partitioned limit 10;

id url article sentiment emotion batch 0f05ec052afe07134371fbee1ac28f13 https://www.techradar.com/computing/software/how-to-right-click-on-a-mac Tools & requirements • System: macOS

• Tools: A working mouse

• Skill level: Beginner

If you don't know how to right-click on a Mac, you're not alone. Back in the olden days of Apple computers, the idea of a separate right-click mouse button would have been strange given the simplicity of the graphical interface, but we've come a long way since then. Unfortunately, Apple has not, at least not directly.

Its best mouse, the Magic Mouse, only has one button, and there's no real obvious way to perform a right-click on macOS. Even the best MacBooks only have a left-click function built into the trackpad.

You'd be forgiven if you thought it just wasn't an option on Macs, but you'd be wrong. It's easy to learn how to right-click on a Mac, and I'll walk you through all the different ways you can do so and take advantage of macOS's context menu, just like you would on Windows and ChromeOS.

Quick steps for how to right-click on a Mac

Use a mouse that has two buttons

Use the control-click function

Step-by-step guide for how to right-click on a Mac - Method one

1. Press and hold the 'Control' key on your keyboard (Image: © Future) Press down on the "Control" key on your Mac keyboard.

2. Click the mouse or trackpad (Image: © Future) While still holding control, Click on either the trackpad of your MacBook or the mouse button on your mouse. A context menu will pop up once you do, its contents depend on what you clicked on or where. That's it!

Step-by-step guide for how to right-click on a Mac - Method two

1. Use a mouse with a right mouse button (Image: © Future) Many of the best mice for Macs have more than one button, and using one of these rather than a one-button mouse is the easiest way to right-click on a Mac. With these mice, it's as simple to right-click as you would on a Windows PC.

2. Setting up your mouse with a Mac (Image: © Future) Typically, your mouse should be detected automatically by your Mac, but if it doesn't, you might need to allow the peripheral permission to connect. When the pop-up appears, click Allow to grant your mouse the necessary permission.

3. Set up your mouse settings by going into system settings (Image: © Future) Click on the Apple icon on the menu bar and select System Settings.

4. Select 'Mouse' from the left-side menu (Image: © Future) Select the 'Mouse' option from the selections on the left side of the system settings window.

5. Set your right-click preferences (Image: © Future) Use the Mouse Settings menu to select your preferred right-click settings. That's it!

Final thoughts

Now that you know how to right-click on a Mac, no matter your setup, you have access to a whole new context menu to improve the functionality of your Mac. You can use this menu to open files or programs, move them to the trash, and much more.

It's as useful on a Mac as the right-click menu is on Windows, and now the difference between the two different operating systems is just a little bit less pronounced. neutral neutral 10 bec71e7e08a0bad943b3d21c7320afee https://www.pulse.com.gh/news/filla/prophet-oduro-lists-ghanaian-churches-to-be-closed-down-check-if-yours-is-part/tvph3xy In a video shared on YouTube by NGOSRA TV GH, the man of God is heard telling his congregation that closing down some of the churches that are practising things that are at variance with the Bible is long overdue.

“There are churches in Ghana where they mix concoctions with things like pomades and fragrance and then sell them out to people that they have certain properties for people and all these are jokes people do in the name of God.

“Silliness is all they are engaged in. The leader can do silly things to the extent that he goes to lie on the floor, dancing to earthly songs in the church. What are you expecting that people learn from the church? If someone is not born again, what do yo

In [0]:
%sql
  -- drop table infosys.kamikaze_rtsa.gold_table

In [0]:
%sql

--  CREATE TABLE infosys.kamikaze_rtsa.gold_table_partitioned (
--    id STRING,
--    url STRING,
--    article STRING,
--   sentiment STRING,
--   emotion STRING
-- )
-- USING DELTA
-- partitioned by (batch int);

 CREATE OR REPLACE TABLE infosys.kamikaze_rtsa.gold_table_streaming (
   id STRING,
   url STRING,
   article STRING,
  sentiment STRING,
  emotion STRING
)
USING DELTA
partitioned by (batch int);

In [0]:
%pip install transformers torch xformers

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Using cached xformers-0.0.21-cp310-cp310-manylinux2014_x86_64.whl (167.0 MB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
  Using cached PyYAML-6.0.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (705 kB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
  Using cached safetensors-0.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached regex-2023.8.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (771 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cusolver_cu11-11.4.0.1-2-py3-none-manylinux1_x86_64.whl (102.6 

In [0]:
dbutils.library.restartPython()

In [0]:
"""Function Definitions and Import"""
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import col, md5, udf, lit, monotonically_increasing_id, expr, pandas_udf
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
from scipy import stats
import numpy as np
import warnings
import pandas as pd
spark = SparkSession.builder.appName("SentimentEmotionPipeline").getOrCreate()
spark.conf.set("spark.databricks.delta.autoOptimize.enabled", "true")
warnings.filterwarnings("ignore")

# Load sentiment analysis model and tokenizer
sentiment_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

# Load emotion analysis model and tokenizer
emotion_tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
emotion_model = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")


def get_sentiment(text, tokenizer=sentiment_tokenizer, model=sentiment_model):
    chunk_size = 512

    # Tokenize the text
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size)

    # Split the tokens into chunks
    num_chunks = (len(tokens['input_ids'][0]) - 1) // chunk_size + 1
    sentiments = []

    # Process each chunk
    for i in range(num_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        chunk_tokens = {key: value[:, start:end] for key, value in tokens.items()}

        # Perform sentiment analysis on the current chunk
        with torch.no_grad():
            outputs = model(**chunk_tokens)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1)
            sentiments.append(predicted_class.item())

    # Map sentiment IDs to labels
    sentiment_labels = [
        "negative", "neutral", "positive"
    ]

    sentiment_labels = np.array(sentiment_labels)
    mapped_sentiments = sentiment_labels[sentiments]

    # Calculate the mode sentiment
    mode_sentiment = stats.mode(mapped_sentiments)[0][0]

    return str(mode_sentiment)


def get_emotion(text, tokenizer=emotion_tokenizer, model=emotion_model):
    chunk_size = 512

    # Tokenize the text
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size)

    # Split the tokens into chunks
    num_chunks = (len(tokens['input_ids'][0]) - 1) // chunk_size + 1
    emotions = []

    # Process each chunk
    for i in range(num_chunks):
        start = i * chunk_size
        end = (i + 1) * chunk_size
        chunk_tokens = {key: value[:, start:end] for key, value in tokens.items()}

        # Perform sentiment analysis on the current chunk
        with torch.no_grad():
            outputs = model(**chunk_tokens)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=1)
            emotions.append(predicted_class.item())

    # Map sentiment IDs to labels
    emotion_labels = [
        "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion",
        "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment",
        "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism",
        "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
    ]

    emotion_labels = np.array(emotion_labels)
    mapped_emotions = emotion_labels[emotions]

    # Calculate the mode sentiment
    mode_emotion = stats.mode(mapped_emotions)[0][0]
    return str(mode_emotion)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [0]:
""" Partioned, Un-Batched Original Pipeline (Slow Prediction, works with Fast-Preferred Cluster)"""
silver_table_df = spark.sql("SELECT batch, url, transformed_text as article from infosys.kamikaze_rtsa.silver_table_processed \
                            --where batch = (select max(batch) from infosys.kamikaze_rtsa.silver_table_processed) \
                            ")\
        .withColumn("id", md5(col("url")))\
        .select(*['id','batch','url','article'])
sentiments = []
emotions = []

for row in silver_table_df.collect():
    sentiment_prediction = get_sentiment(row["article"])
    emotion_prediction = get_emotion(row["article"])
    sentiments.append(sentiment_prediction)
    emotions.append(emotion_prediction)    

predictions_df = spark.createDataFrame(zip(sentiments, emotions), ["sentiment", "emotion"])
# Add a sequential index column to both DataFrames to use for joining
silver_table_df = silver_table_df.withColumn("join_key", lit(1))
predictions_df = predictions_df.withColumn("join_key", lit(1))

# Join the two DataFrames on the temporary join_key column
merged_df = silver_table_df.join(predictions_df, on="join_key").drop("join_key")
merged_df.select("id", "url", "article", "sentiment", "emotion","batch").write.mode("append").format("delta").partitionBy("batch").saveAsTable("infosys.kamikaze_rtsa.gold_table")

In [0]:
""" Partitioned, Batched UDF Pipeline (Slow Write, works with Custom Cluster)"""
get_sentiment_udf = udf(get_sentiment, StringType())
get_emotion_udf = udf(get_emotion, StringType())
silver_table_df = spark.sql("SELECT batch, url, transformed_text as article from infosys.kamikaze_rtsa.silver_table_processed \
                             where batch = (select max(batch) from infosys.kamikaze_rtsa.silver_table_processed) \
                             ")\
        .withColumn("id", md5(col("url")))\
        .select(*['id','batch','url','article'])
unique_batches = silver_table_df.select("batch").distinct().collect()
for batch_row in unique_batches:    
    batch_number = batch_row["batch"]    
    batch_df = silver_table_df.filter(col("batch") == batch_number)
    print(f"Executing Batch {batch_number} with {batch_df.count()} rows")
    # Process sentiment and emotion
    batch_df = batch_df.withColumn("sentiment", lit(get_sentiment_udf(batch_df.article)))
    batch_df = batch_df.withColumn("emotion", lit(get_emotion_udf(batch_df.article)))    
    
    
    # Write the processed data to the gold table
    batch_df.select("id", "url", "article", "sentiment", "emotion","batch").write.mode("append").format("delta").partitionBy("batch").saveAsTable("infosys.kamikaze_rtsa.gold_table_partitioned")

Executing Batch 31 with 475 rows
Executing Batch 34 with 475 rows
Executing Batch 28 with 447 rows
Executing Batch 26 with 461 rows
Executing Batch 27 with 458 rows
Executing Batch 44 with 467 rows
Executing Batch 12 with 452 rows
Executing Batch 22 with 478 rows
Executing Batch 47 with 476 rows
Executing Batch 1 with 1330 rows
Executing Batch 13 with 470 rows
Executing Batch 6 with 473 rows
Executing Batch 16 with 467 rows
Executing Batch 3 with 473 rows
Executing Batch 20 with 478 rows
Executing Batch 40 with 470 rows
Executing Batch 54 with 471 rows
Executing Batch 48 with 467 rows
Executing Batch 5 with 452 rows
Executing Batch 19 with 466 rows
Executing Batch 41 with 469 rows
Executing Batch 43 with 473 rows
Executing Batch 15 with 482 rows
Executing Batch 37 with 473 rows
Executing Batch 9 with 461 rows
Executing Batch 17 with 478 rows
Executing Batch 35 with 463 rows
Executing Batch 4 with 469 rows
Executing Batch 8 with 470 rows
Executing Batch 23 with 480 rows
Executing Batch 

In [0]:
# """Batched Original Pipeline (Slow Prediction, works with Fast-Preferred Cluster)"""
# """ Faulty, Creates Duplicates """
# silver_table_df = spark.sql("SELECT batch, url, transformed_text as article from infosys.kamikaze_rtsa.silver_table_processed \
#                             --where batch = (select max(batch) from infosys.kamikaze_rtsa.silver_table_processed) \
#                             ")\
#         .withColumn("id", md5(col("url")))\
#         .select(*['id','batch','url','article'])
# unique_batches = silver_table_df.select("batch").distinct().collect()
# for batch_row in unique_batches:
#     batch_number = batch_row["batch"]
#     print(f"Executing Batch {batch_number}")
#     batch_df = silver_table_df.filter(col("batch") == batch_number)
#     sentiments = []
#     emotions = []

#     for row in batch_df.collect():
#         sentiment_prediction = get_sentiment(row["article"])
#         emotion_prediction = get_emotion(row["article"])
#         sentiments.append(sentiment_prediction)
#         emotions.append(emotion_prediction)    

#     predictions_df = spark.createDataFrame(zip(sentiments, emotions), ["sentiment", "emotion"])
#     # Add a sequential index column to both DataFrames to use for joining
#     batch_df = batch_df.withColumn("join_key", lit(1))
#     predictions_df = predictions_df.withColumn("join_key", lit(1))

#     # Join the two DataFrames on the temporary join_key column
#     merged_df = batch_df.join(predictions_df, on="join_key").drop("join_key")
#     merged_df.select("id", "url", "article", "sentiment", "emotion").write.mode("append").format("delta").saveAsTable("infosys.kamikaze_rtsa.gold_table")

In [0]:
""" Streaming Pipeline (Requires UDF support)"""
"""POC"""
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import StringType
from pyspark.sql.functions import md5, col
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("StreamingUDFPipelineWithProgress") \
    .getOrCreate()


# Define UDFs
get_sentiment_udf = udf(get_sentiment, StringType())
get_emotion_udf = udf(get_emotion, StringType())

# Read streaming data using structured streaming
streaming_df = spark.readStream.table("infosys.kamikaze_rtsa.silver_table_processed") \
    .select("batch", "url", "transformed_text") \
    .withColumn("id", md5(col("url"))) \
    .select(*['id', 'batch', 'url', 'transformed_text']).where(col('batch')==74)

# Process streaming data
processed_streaming_df = streaming_df \
    .withColumn("sentiment", get_sentiment_udf(streaming_df["transformed_text"])) \
    .withColumn("emotion", get_emotion_udf(streaming_df["transformed_text"]))

# Write processed data to Delta Lake in append mode
query = processed_streaming_df.writeStream \
    .outputMode("append") \
    .foreachBatch(lambda batch_df, batch_id: batch_df
                  .select("id", "url", "transformed_text", "sentiment", "emotion", "batch")
                  .write
                  .mode("append")
                  .option("mergeSchema", "true")
                  .format("delta").partitionBy("batch")  
                  .saveAsTable("infosys.kamikaze_rtsa.gold_table_streaming")) \
    .start()

# Wait for the query to terminate
query.awaitTermination()
      

In [0]:
# """ Unpartitioned Batched UDF Pipeline (Slow Write, works with Custom Cluster)"""
# get_sentiment_udf = udf(get_sentiment, StringType())
# get_emotion_udf = udf(get_emotion, StringType())
# silver_table_df = spark.sql("SELECT batch, url, transformed_text as article from infosys.kamikaze_rtsa.silver_table_processed \
#                              --where batch = (select max(batch) from infosys.kamikaze_rtsa.silver_table_processed) \
#                              ")\
#         .withColumn("id", md5(col("url")))\
#         .select(*['id','batch','url','article'])
# unique_batches = silver_table_df.select("batch").distinct().collect()
# for batch_row in unique_batches:    
#     batch_number = batch_row["batch"]    
#     batch_df = silver_table_df.filter(col("batch") == batch_number)
#     print(f"Executing Batch {batch_number} with {batch_df.count()} rows")
#     # Process sentiment and emotion
#     batch_df = batch_df.withColumn("sentiment", lit(get_sentiment_udf(batch_df.article)))
#     batch_df = batch_df.withColumn("emotion", lit(get_emotion_udf(batch_df.article)))    
    
    
#     # Write the processed data to the gold table
#     batch_df.select("id", "url", "article", "sentiment", "emotion").write.mode("append").format("delta").saveAsTable("infosys.kamikaze_rtsa.gold_table")

In [0]:
# """Chunkify Pipeline (Faulty as it creates a lot of dupicates)"""
# def chunkify(df: F.DataFrame, chunk_size: int):
#     start = 0
#     length = df.count()

#     # If DF is smaller than the chunk, return the DF
#     if length <= chunk_size:
#         yield df
#         return

#     # Yield individual chunks
#     while start + chunk_size <= length:
#         yield df.take(chunk_size)
#         start = start + chunk_size

#     # Yield the remainder chunk, if needed
#     if start < length:
#         yield df.take(length - start)

# def load_data_score():
    
#     urlarticledf = spark.sql("SELECT url, transformed_text as article from infosys.kamikaze_rtsa.silver_table_processed \
#                             --where batch = (select max(batch) from infosys.kamikaze_rtsa.silver_table_processed) \
#                             ")\
#         .withColumn("id", md5(col("url")))\
#         .select(*['id','url','article'])
#     chunks = chunkify(urlarticledf,500)
#     return(chunks)

# def score_emotion_sentiment(urlarticledf):
#     """Original Pipeline"""
#     sentiments = []
#     emotions = []

#     for row in urlarticledf.collect():
#         sentiment_prediction = get_sentiment(row["article"])
#         emotion_prediction = get_emotion(row["article"])
#         sentiments.append(sentiment_prediction)
#         emotions.append(emotion_prediction)    

#     predictions_df = spark.createDataFrame(zip(sentiments, emotions), ["sentiment", "emotion"])
#     # Add a sequential index column to both DataFrames to use for joining
#     urlarticledf = urlarticledf.withColumn("join_key", lit(1))
#     predictions_df = predictions_df.withColumn("join_key", lit(1))

#     # Join the two DataFrames on the temporary join_key column
#     merged_df = urlarticledf.join(predictions_df, on="join_key").drop("join_key")
#     merged_df.write.mode("append").format("delta").saveAsTable("infosys.kamikaze_rtsa.gold_table")

# chunks = load_data_score()
# for i,chunk in enumerate(chunks):
#     print(i)
#     df = spark.createDataFrame(data=chunk)
#     score_emotion_sentiment(df)